<a href="https://colab.research.google.com/github/Justy-11/algorithms/blob/main/TwoPassAlgorithmForImageSegmentation/TwoPassAlgorithmForImageSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import random

def two_pass_algo(binary_image):
    # Get the dimensions of the input image
    height, width = binary_image.shape

    # Create a zero matrix with the same dimensions
    # for labels
    labeled_image = np.zeros((height, width), dtype=np.int16)

    # for assigning new labels
    label = 0

    # dictionary for similarity map
    equivalence_dict = {}

    # first pass
    for i in range(height):
        for j in range(width):

          Current_pixel_color = binary_image[i][j]

          # first pixel - doesn't have North or West pixel
          if i == 0 and j == 0:
            labeled_image[i][j] = label
            label = label + 1
          # first row - only have west pixel
          if i == 0 and not (j == 0):
            if Current_pixel_color == binary_image[i][j-1]:
              labeled_image[i][j] = labeled_image[i][j-1]
            else:
              labeled_image[i][j] = label
              label = label + 1
          # first column - only have north pixel
          if not (i == 0) and j == 0:
            if Current_pixel_color == binary_image[i-1][j]:
              labeled_image[i][j] = labeled_image[i-1][j]
            else:
              labeled_image[i][j] = label
              label = label + 1
          # rest of the pixels
          if not (i == 0) and not (j == 0):
            West_pixel_color = binary_image[i][j-1]
            North_pixel_color = binary_image[i-1][j]
            West_pixel_label = labeled_image[i][j-1]
            North_pixel_label = labeled_image[i-1][j]

            if (Current_pixel_color == West_pixel_color) and (Current_pixel_color == North_pixel_color) and not (West_pixel_label == North_pixel_label):
              labeled_image[i][j] = min(West_pixel_label, North_pixel_label)

              if not (West_pixel_label == North_pixel_label):
                equivalence_dict[max(West_pixel_label, North_pixel_label)] = min(West_pixel_label, North_pixel_label)

            if (Current_pixel_color == West_pixel_color) and (Current_pixel_color == North_pixel_color) and (West_pixel_label == North_pixel_label):
              labeled_image[i][j] = West_pixel_label

            if (Current_pixel_color == North_pixel_color) and not (Current_pixel_color == West_pixel_color):
              labeled_image[i][j] = North_pixel_label

            if (Current_pixel_color == West_pixel_color) and not (Current_pixel_color == North_pixel_color):
              labeled_image[i][j] = West_pixel_label

            if not (Current_pixel_color == North_pixel_color) and not (Current_pixel_color == West_pixel_color):
              labeled_image[i][j] = label
              label = label + 1

    # second pass
    for i in range(height):
      for j in range(width):
        label = labeled_image[i][j]
        while label in equivalence_dict:
          label = equivalence_dict[label]
          labeled_image[i][j] = label

    return labeled_image

# Load the sample binary image - update with the image path
binary_image = cv2.imread('Sample Image 1.bmp', cv2.IMREAD_GRAYSCALE)

# Convert 255 to 1
binary_image[binary_image == 255] = 1

labeled_image = two_pass_algo(binary_image)

colors = {}
# take only unique lables
unique_labels = np.unique(labeled_image)

height, width = binary_image.shape

for label in unique_labels[1:]:
  colors[label] = (random.randint(0, 255),
                   random.randint(0, 255),
                   random.randint(0, 255))
  output_image = np.zeros((height, width, 3), dtype=np.uint8)

for label in unique_labels[1:]:
  output_image[labeled_image == label] = colors[label]

# Save the colored output image
cv2.imwrite('output_image.png', output_image)

True

In [ ]:
labeled_image.shape, binary_image.shape
# # Save the colored output image
# cv2.imwrite('output_colored_image.png', two_pass_algo(binary_image))

((447, 640), (447, 640))

In [ ]:
np.max(labeled_image)

178